In [28]:
import requests
import pandas as pd
import json 
import time
import numpy as np 

In [45]:
# date_range = pd.date_range('2022-03-18', '2022-03-20')
# date_range = pd.date_range('2022-01-01', '2024-10-30')
# date_range = pd.date_range('2023-04-29', '2024-10-30') # 위 기간 실행 중 2023.4.29에 오류 발생해서 28일까지 데이터는 파일로 저장하고 다시 실행
date_range = pd.date_range('2023-06-27', '2024-10-30')  # 위 기간 실행 중 2023.6.26에 오류 발생해서 25일까지 데이터는 파일로 저장하고 다시 실행
                                                        # 2023.06.26 데이터는 없음 - 27일 데이터부터 조회
# print(date_range)

service_key = '445754734a6c656f363548676f6665'
start, end = 1, 1000
all_df_bikes = []
for date in date_range:
    
    print("data at {0} will be processed".format(date))

    while True:        
        date_string = date.strftime('%Y-%m-%d')
        url = 'http://openapi.seoul.go.kr:8088/{0}/json/tbCycleUseStatus/{1}/{2}/{3}'.format(service_key, start, end, date_string)

        # API 요청
        response = requests.get(url)

        # 응답 상태 확인
        if response.status_code == 200:

            try:
                # 요청이 성공했을 경우 내용 출력
                # print(response.content)
                decoded_response_content = response.content.decode('utf-8')

                dict_content = json.loads(decoded_response_content) 

                df_bike = pd.DataFrame(dict_content['useStatus']['row'])
                all_df_bikes.append(df_bike)

                total_count = int(dict_content['useStatus']['list_total_count'])
                if total_count > end:
                    start = end + 1
                    if total_count > end + 1000:
                        end = end + 1000
                    else:
                        end = total_count
                else:
                    start, end = 1, 1000
                    break

            except Exception as e:
                print(e)
                time.sleep(1)                
                
        else:
            # 오류가 발생했을 경우 상태 코드 출력
            print(f"Request failed with status code: {response.status_code}")




data at 2023-06-27 00:00:00 will be processed
data at 2023-06-28 00:00:00 will be processed
data at 2023-06-29 00:00:00 will be processed
data at 2023-06-30 00:00:00 will be processed
data at 2023-07-01 00:00:00 will be processed
data at 2023-07-02 00:00:00 will be processed
data at 2023-07-03 00:00:00 will be processed
data at 2023-07-04 00:00:00 will be processed
data at 2023-07-05 00:00:00 will be processed
data at 2023-07-06 00:00:00 will be processed
data at 2023-07-07 00:00:00 will be processed
data at 2023-07-08 00:00:00 will be processed
data at 2023-07-09 00:00:00 will be processed
data at 2023-07-10 00:00:00 will be processed
data at 2023-07-11 00:00:00 will be processed
data at 2023-07-12 00:00:00 will be processed
data at 2023-07-13 00:00:00 will be processed
data at 2023-07-14 00:00:00 will be processed
data at 2023-07-15 00:00:00 will be processed
data at 2023-07-16 00:00:00 will be processed
data at 2023-07-17 00:00:00 will be processed
data at 2023-07-18 00:00:00 will b

In [46]:
len(all_df_bikes)
all_df_bikes[-2]

,STA_LOC,RENT_NM,STAT_DATA,RENT_CNT,RTN_CNT,START_INDEX,END_INDEX,RNUM
0,용산구,853.용산역 맞은편,2024-10-30,145,155,0,0,1001
1,용산구,854.HID 유족동지회 앞,2024-10-30,5,4,0,0,1002
2,용산구,855.이촌역5번출구 앞,2024-10-30,7,6,0,0,1003
3,서대문구,161. 무악재역1번 출구,2024-10-30,4,10,0,0,1004
4,용산구,859.남산공원 야외식물원 앞,2024-10-30,2,2,0,0,1005
...,...,...,...,...,...,...,...,...
995,강동구,4953. 세양청마루아파트 앞,2024-10-30,54,67,0,0,1996
996,용산구,4643. 한가람아파트 앞,2024-10-30,23,33,0,0,1997
997,영등포구,5871. 건영상가 앞,2024-10-30,32,31,0,0,1998
998,강서구,5096. LG사이언스파크 E14동,2024-10-30,262,327,0,0,1999


In [48]:
len(all_df_bikes)

1476

In [49]:
merged_df_bikes = pd.concat(all_df_bikes)

In [51]:
print( merged_df_bikes.shape )
merged_df_bikes.head()

(1327876, 8)


,STA_LOC,RENT_NM,STAT_DATA,RENT_CNT,RTN_CNT,START_INDEX,END_INDEX,RNUM
0,마포구,108. 서교동 사거리,2023-06-27,86,86,0,0,1
1,양천구,729. 서부식자재마트 건너편,2023-06-27,85,101,0,0,2
2,양천구,731. 서울시 도로환경관리센터,2023-06-27,158,174,0,0,3
3,양천구,732. 신월중학교,2023-06-27,68,69,0,0,4
4,양천구,733. 신정이펜하우스314동,2023-06-27,33,18,0,0,5


In [52]:
merged_df_bikes.to_csv("../bike-data/bike-data-20230627-20241030.csv", index=False)

In [34]:
import requests

service_key = '445754734a6c656f363548676f6665'
url = f'http://openapi.seoul.go.kr:8088/{service_key}/json/tbCycleUseStatus/1/1000/2022-01-05'

# API 요청
response = requests.get(url)

# 응답 상태 확인
if response.status_code == 200:
    # 요청이 성공했을 경우 내용 출력
    print(response.content)
else:
    # 오류가 발생했을 경우 상태 코드 출력
    print(f"Request failed with status code: {response.status_code}")

b'{"useStatus":{"list_total_count":"2547","RESULT":{"CODE":"INFO-000","MESSAGE":"\xec\xa0\x95\xec\x83\x81 \xec\xb2\x98\xeb\xa6\xac\xeb\x90\x98\xec\x97\x88\xec\x8a\xb5\xeb\x8b\x88\xeb\x8b\xa4."},"row":[{"STA_LOC":"\xeb\xa7\x88\xed\x8f\xac\xea\xb5\xac","RENT_NM":"108. \xec\x84\x9c\xea\xb5\x90\xeb\x8f\x99 \xec\x82\xac\xea\xb1\xb0\xeb\xa6\xac","STAT_DATA":"2022-01-05","RENT_CNT":"34","RTN_CNT":"42","START_INDEX":0,"END_INDEX":0,"RNUM":"1"},{"STA_LOC":"\xec\x96\x91\xec\xb2\x9c\xea\xb5\xac","RENT_NM":"729. \xec\x84\x9c\xeb\xb6\x80\xec\x8b\x9d\xec\x9e\x90\xec\x9e\xac\xeb\xa7\x88\xed\x8a\xb8 \xea\xb1\xb4\xeb\x84\x88\xed\x8e\xb8","STAT_DATA":"2022-01-05","RENT_CNT":"50","RTN_CNT":"53","START_INDEX":0,"END_INDEX":0,"RNUM":"2"},{"STA_LOC":"\xec\x96\x91\xec\xb2\x9c\xea\xb5\xac","RENT_NM":"731. \xec\x84\x9c\xec\x9a\xb8\xec\x8b\x9c \xeb\x8f\x84\xeb\xa1\x9c\xed\x99\x98\xea\xb2\xbd\xea\xb4\x80\xeb\xa6\xac\xec\x84\xbc\xed\x84\xb0","STAT_DATA":"2022-01-05","RENT_CNT":"40","RTN_CNT":"46","START_INDEX":0,

In [35]:
df_list2 = []

# 여러 개의 파일에서 데이터 읽기 + 데이터프레임에 저장 + 리스트에 저장

for ym in ["20220101-20230428", "20230429-20230625", "20230627-20241030"]:
    print('../bike-data/bike-data-{0}.csv'.format(ym))
    df = pd.read_csv('../bike-data/bike-data-{0}.csv'.format(ym), encoding='utf-8')
    df_list2.append(df)

../bike-data/bike-data-20220101-20230428.csv
../bike-data/bike-data-20230429-20230625.csv
../bike-data/bike-data-20230627-20241030.csv


In [1]:
import pandas as pd

df_list2 = []

# 여러 개의 파일에서 데이터 읽기 + 데이터프레임에 저장 + 리스트에 저장
for ym in ["20220101-20230428", "20230429-20230625", "20230627-20241030"]:
    file_path = f'../bike-data/bike-data-{ym}.csv'  # 파일 경로
    print(f"Reading file: {file_path}")  # 파일 경로 출력
    
    # 파일을 읽어서 데이터프레임으로 저장
    df = pd.read_csv(file_path, encoding='utf-8')
    
    # 데이터프레임 리스트에 추가
    df_list2.append(df)

# 데이터프레임 리스트를 하나로 합침
merged_df_bikes = pd.concat(df_list2, ignore_index=True)

# 병합된 데이터프레임 확인
merged_df_bikes.head()

# 병합된 데이터프레임을 CSV로 저장
merged_df_bikes.to_csv('../bike-data/merged_bike_data.csv', index=False)


Reading file: ../bike-data/bike-data-20220101-20230428.csv
Reading file: ../bike-data/bike-data-20230429-20230625.csv
Reading file: ../bike-data/bike-data-20230627-20241030.csv


In [36]:
for ym in []:
    df = pd.read_csv('../bike-')

In [ ]:
merged_df_bikes.to_csv('../bike/merged_bikes_data.csv', index=False)

In [38]:
# df_list
# df_list에 들어있는 여러 개의 dataframe을 행 방향으로 합치기
# merged_df_bikes = pd.concat(df_list2) 

In [ ]:
merged_df_bikes

,STA_LOC,RENT_NM,STAT_DATA,RENT_CNT,RTN_CNT,START_INDEX,END_INDEX,RNUM
0,마포구,108. 서교동 사거리,2022-03-18,34,45,0,0,1
1,양천구,729. 서부식자재마트 건너편,2022-03-18,51,63,0,0,2
2,양천구,731. 서울시 도로환경관리센터,2022-03-18,48,43,0,0,3
3,양천구,732. 신월중학교,2022-03-18,26,27,0,0,4
4,양천구,733. 신정이펜하우스314동,2022-03-18,27,11,0,0,5
...,...,...,...,...,...,...,...,...
584,양천구,722. 경서농협 버스정류장(우리은행신정동지점방향),2022-03-20,28,31,0,0,2585
585,양천구,723. SBS방송국,2022-03-20,42,48,0,0,2586
586,양천구,724. 계남공원 입구 주출입구 좌측,2022-03-20,14,12,0,0,2587
587,양천구,725. 양강중학교앞 교차로,2022-03-20,18,4,0,0,2588


In [ ]:
# 컬럼명 바꾸기
merged_df_bikes.rename(columns={'STA_LOC':'지역명','RENT_MN':'대여소명','STAT_DATA':'대여일자','RENT_CNT':'대여건수','RNT_CNT':'반납건수','RNUM':'순번'}, inplace=True)

In [ ]:
# 안바뀐 컬럼명 바꾸기
merged_df_bikes.rename(columns={'RENT_NM':'대여소명','RTN_CNT':'반납건수'}, inplace=True)

In [ ]:
merged_df_bikes.columns

Index(['지역명', '대여소명', '대여일자', '대여건수', '반납건수', 'START_INDEX', 'END_INDEX',
       '순번'],
      dtype='object')

In [ ]:
merged_df_bikes.dtypes

지역명            object
대여소명           object
대여일자           object
대여건수            int64
반납건수            int64
START_INDEX     int64
END_INDEX       int64
순번              int64
dtype: object

In [ ]:
import pandas as pd 

df = pd.read_csv('combined_data.csv', encoding='utf-8')

df.head()

,일시,주말_공휴일,공휴일
0,2022-01-01,1,주말
1,2022-01-02,1,주말
2,2022-01-03,0,평일
3,2022-01-04,0,평일
4,2022-01-05,0,평일


In [ ]:
# 대여건수 열을 숫자로 변환
merged_df_bikes['대여건수'] = pd.to_numeric(merged_df_bikes['대여건수'], errors='coerce')

# 일자별 대여건수 (float)
daily_rentals = merged_df_bikes.groupby('대여일자')['대여건수'].mean()
daily_rentals=daily_rentals.reset_index()

# 일자별 대여건수 (int)로 바꾸기
daily_rentals["대여건수"]=daily_rentals["대여건수"].astype(int)

daily_rentals

NameError: name 'merged_df_bikes' is not defined